In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf


import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import yfinance as yf
yf.pdr_override()

def find_election_day(year):
    # The election is in November
    month = 11
    # Start with the first of November
    date = datetime(year, month, 1)
    # Find the first Tuesday. If the first of November is not a Tuesday, move to the next Tuesday
    while date.weekday() != 1:  # Monday is 0, Tuesday is 1, etc.
        date += timedelta(days=1)
    # The election day is the first Tuesday after the first Monday, so add 7 days if we're on the first day
    if date.day == 1:
        date += timedelta(days=7)
    return date

# Years when presidential elections were held, starting from 1789 to 2024, every 4 years
years = list(range(2008, 2024, 4))

# Create a list of election dates
election_dates = [find_election_day(year) for year in years]

# Create a DataFrame
df_elections = pd.DataFrame({
    'Year': years,
    'Election Day': election_dates
})

df_elections.head(3)

,Year,Election Day
0,2008,2008-11-04
1,2012,2012-11-06
2,2016,2016-11-08


In [7]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the time windows
time_windows = {
    '1970_to_today': ('1970-01-01', '2024-06-22'),
    '2008_to_today': ('2008-01-01', '2024-06-22')
}

# Define the election years, post-election years, and non-election years
election_years = [year for year in range(1970, 2024, 4)]
post_election_years = [year + 1 for year in election_years]
all_years = list(range(1970, 2024))
non_election_years = [year for year in all_years if year not in election_years and year not in post_election_years]

# Download S&P 500 data
symbol = '^GSPC'

# Prepare the data
def prepare_data(start_date, end_date, filter_years=None):
    df = yf.download(symbol, start=start_date, end=end_date)
    df['Daily Return'] = df['Adj Close'].pct_change() * 100  # Convert to percentage
    df = df.dropna()
    df['Day of Year'] = df.index.day_of_year
    if filter_years is not None:
        df = df[df.index.year.isin(filter_years)]
    daily_returns_aggregated = df.groupby('Day of Year')['Daily Return'].mean().reset_index()
    daily_returns_aggregated['Color'] = daily_returns_aggregated['Daily Return'].apply(lambda x: 'green' if x > 0 else 'red')
    return daily_returns_aggregated

# Prepare data for each category
data_1970_to_today_all_years = prepare_data(*time_windows['1970_to_today'], filter_years=None)
data_1970_to_today_election_years = prepare_data(*time_windows['1970_to_today'], filter_years=election_years)
data_2008_to_today_election_years = prepare_data(*time_windows['2008_to_today'], filter_years=election_years)
data_post_election_years = prepare_data(*time_windows['1970_to_today'], filter_years=post_election_years)
data_1970_to_today_non_election_years = prepare_data(*time_windows['1970_to_today'], filter_years=non_election_years)

# Generate the month labels for the x-axis
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
month_end_positions = [sum(month_days[:i+1]) for i in range(12)]
week_positions = list(range(7, 366, 7))

# Determine the common y-axis range
y_min = min(data_1970_to_today_all_years['Daily Return'].min(), 
            data_1970_to_today_election_years['Daily Return'].min(), 
            data_2008_to_today_election_years['Daily Return'].min(), 
            data_post_election_years['Daily Return'].min(),
            data_1970_to_today_non_election_years['Daily Return'].min())
y_max = max(data_1970_to_today_all_years['Daily Return'].max(), 
            data_1970_to_today_election_years['Daily Return'].max(), 
            data_2008_to_today_election_years['Daily Return'].max(), 
            data_post_election_years['Daily Return'].max(),
            data_1970_to_today_non_election_years['Daily Return'].max())

# Create subplots
fig = make_subplots(
    rows=5, cols=1,
    subplot_titles=(
        '1970 to Today (All Years)', 
        '1970 to Today (Election Years)',
        '2008 to Today (Election Years)', 
        'Post-Election Years', 
        '1970 to Today (Non-Election Years)'),
    vertical_spacing=0.02  # Reduce the space between the plots
)

# Add bars for each category
fig.add_trace(go.Bar(
    x=data_1970_to_today_all_years['Day of Year'],
    y=data_1970_to_today_all_years['Daily Return'],
    marker_color=data_1970_to_today_all_years['Color'],
    name='1970 to Today (All Years)'
), row=1, col=1)

fig.add_trace(go.Bar(
    x=data_1970_to_today_election_years['Day of Year'],
    y=data_1970_to_today_election_years['Daily Return'],
    marker_color=data_1970_to_today_election_years['Color'],
    name='1970 to Today (Election Years)'
), row=2, col=1)

fig.add_trace(go.Bar(
    x=data_2008_to_today_election_years['Day of Year'],
    y=data_2008_to_today_election_years['Daily Return'],
    marker_color=data_2008_to_today_election_years['Color'],
    name='2008 to Today (Election Years)'
), row=3, col=1)

fig.add_trace(go.Bar(
    x=data_post_election_years['Day of Year'],
    y=data_post_election_years['Daily Return'],
    marker_color=data_post_election_years['Color'],
    name='Post-Election Years'
), row=4, col=1)

fig.add_trace(go.Bar(
    x=data_1970_to_today_non_election_years['Day of Year'],
    y=data_1970_to_today_non_election_years['Daily Return'],
    marker_color=data_1970_to_today_non_election_years['Color'],
    name='1970 to Today (Non-Election Years)'
), row=5, col=1)

# Add vertical lines for the end of each month
for position in month_end_positions:
    fig.add_vline(x=position, line=dict(color='blue', width=1, dash='dash'))

# Add small ticks to denote the end of each week
for week_position in week_positions:
    fig.add_vline(x=week_position, line=dict(color='black', width=0.5, dash='dot'))

# Update layout with month labels, week ticks, and shared y-axis range
fig.update_layout(
    title='Average Daily Returns Aggregated by Day of Year',
    xaxis1=dict(tickmode='array', tickvals=month_end_positions, ticktext=month_labels, showgrid=True, tick0=0, dtick=7),
    xaxis2=dict(tickmode='array', tickvals=month_end_positions, ticktext=month_labels, showgrid=True, tick0=0, dtick=7),
    xaxis3=dict(tickmode='array', tickvals=month_end_positions, ticktext=month_labels, showgrid=True, tick0=0, dtick=7),
    xaxis4=dict(tickmode='array', tickvals=month_end_positions, ticktext=month_labels, showgrid=True, tick0=0, dtick=7),
    xaxis5=dict(tickmode='array', tickvals=month_end_positions, ticktext=month_labels, showgrid=True, tick0=0, dtick=7),
    yaxis1=dict(range=[y_min, y_max]),
    yaxis2=dict(range=[y_min, y_max]),
    yaxis3=dict(range=[y_min, y_max]),
    yaxis4=dict(range=[y_min, y_max]),
    yaxis5=dict(range=[y_min, y_max]),
    yaxis_title='Average Daily Return (%)',
    template='plotly_white',
    showlegend=False,
    height=1500  # Increase the height to make the graphs larger
)

# Show the plot
fig.show()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
